# CSP con Backtracking, Beam y Local Search
# Condiciones del problema:
### Se implementará 3 algoritmos diferentes de satisfacción de restricciones para resovler un problema de programación de exámenes para cuatro estudiantes que toman 7 exámenes diferentes. El problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.
- Todos los exámenes deberán realizarse en días diferentes, concretamente, de lunes a miércoles.
- Ningún estudiante deberá tener más de un examen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

# Definición de variables y estructura del problema

In [9]:

from constraint import Problem




def setup():
    # crear las variables a usar
    exams = ['exam1', 'exam2', 'exam3', 'exam4', 'exam5', 'exam6', 'exam7']
    days = ['monday', 'tuesday', 'wednesday']
    # crear instancia del problema CSP
    problem = Problem()

    # agregar las variables 
    for exam in exams:
        problem.addVariable(exam, days)

    # definimos por ejemplo, los exámenes de cada alumno
    student_exams = {
        'student1': ['exam1', 'exam2'],
        'student2': ['exam3', 'exam4'],
        'student3': ['exam5', 'exam6'],
        'student4': ['exam7']
    }

    # definimos las condiciones 
    for student, student_exams in student_exams.items():
        for exam1 in student_exams:
            for exam2 in student_exams:
                if exam1 != exam2:
                    problem.addConstraint(lambda e1, e2: e1 != e2, (exam1, exam2))

    return problem


## Backtracking

In [10]:
# getSolutions de problem ya utiliza Backtracking
def backtracking(problem):
    return problem.getSolutions()

problem = setup()
solutions = backtracking(problem)

if solutions:
    for solution in solutions:
        print(solution)
else:
    print("Sin soluciones")

{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'tuesday', 'exam7': 'wednesday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'tuesday', 'exam7': 'tuesday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'tuesday', 'exam7': 'monday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'monday', 'exam7': 'wednesday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'monday', 'exam7': 'tuesday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'wednesday', 'exam6': 'monday', 'exam7': 'monday'}
{'exam1': 'wednesday', 'exam2': 'tuesday', 'exam3': 'wednesday', 'exam4': 'tuesday', 'exam5': 'tuesday', 'exam6': 'wedn

## Beam

In [22]:
# empleo de algoritmo beam
def is_valid_assignment(assignment, student_exams):
    """
    Check if an assignment violates any constraints.
    """
    for student, exams in student_exams.items():
        days = [assignment[exam] for exam in exams if exam in assignment]
        if len(days) != len(set(days)):  # Check for duplicate days
            print(f"Invalid for {student} with exams {exams} on days {days}")
            return False
    return True

def generate_successors(assignment, exams, days):
    """
    Generate all possible successors of an assignment by assigning a day to the next unassigned exam,
    with a slight preference for days not heavily used.
    """
    successors = []
    day_usage = {day: sum(1 for d in assignment.values() if d == day) for day in days}

    for exam in exams:
        if exam not in assignment:
            # Sort days by current usage to slightly prioritize less used days
            sorted_days = sorted(days, key=lambda day: day_usage[day])
            for day in sorted_days:
                new_assignment = assignment.copy()
                new_assignment[exam] = day
                successors.append(new_assignment)
            break  # Only assign one exam to generate the successors
    return successors

def heuristic(assignment, student_exams):
    """
    Define a heuristic function that evaluates how "good" a partial assignment is.
    """
    # As a simple heuristic, count the number of exams successfully scheduled without conflicts.
    return sum(1 for exam, day in assignment.items() if is_valid_assignment({exam: day}, student_exams))

def beam_search(student_exams, exams, days, beam_width):
    """
    Perform the beam search algorithm to find an exam schedule that satisfies the constraints.
    """
    beam = [{}]  # Start with an empty assignment
    solutions = []

    while beam:
        next_beam = []
        for assignment in beam:
            if len(assignment) == len(exams):  # Check if all exams have been assigned
              
                if is_valid_assignment(assignment, student_exams):
                    
                    solutions.append(assignment)
            
            
            else:
                # Generate successors and add them to the next_beam
                successors = generate_successors(assignment, exams, days)
                next_beam.extend(successors)

        # Evaluate all successors and select the best ones based on the heuristic
        next_beam.sort(key=lambda a: heuristic(a, student_exams), reverse=True)
        beam = next_beam[:beam_width]  # Only keep the top beam_width assignments

    return solutions  # If the loop ends without returning, no solution was found

# Example usage:
student_exams = {
    'Student1': ['Exam1', 'Exam2'],
    'Student2': ['Exam3', 'Exam4'],
    'Student3': ['Exam5', 'Exam6'],
    'Student4': ['Exam7']
}
exams = ['Exam1', 'Exam2', 'Exam3', 'Exam4', 'Exam5', 'Exam6', 'Exam7']
days = ['Monday', 'Tuesday', 'Wednesday']
beam_width = 3

solution = beam_search(student_exams, exams, days, beam_width)
if solution:
    for sol in solution:
        print(sol)

else:
    print("No solution")




{'Exam1': 'Monday', 'Exam2': 'Tuesday', 'Exam3': 'Wednesday', 'Exam4': 'Monday', 'Exam5': 'Tuesday', 'Exam6': 'Wednesday', 'Exam7': 'Monday'}
{'Exam1': 'Monday', 'Exam2': 'Tuesday', 'Exam3': 'Wednesday', 'Exam4': 'Monday', 'Exam5': 'Tuesday', 'Exam6': 'Wednesday', 'Exam7': 'Tuesday'}
{'Exam1': 'Monday', 'Exam2': 'Tuesday', 'Exam3': 'Wednesday', 'Exam4': 'Monday', 'Exam5': 'Tuesday', 'Exam6': 'Wednesday', 'Exam7': 'Wednesday'}


## Local Search